In [6]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics
from bayesian_player_ratings import *
import glicko2

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

import trueskillthroughtime as ttt

In [2]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary['Season'] = match_summary['Match_ID'].apply(lambda x: int(x[:4]))

player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats = create_additional_player_stats_variables(player_stats)
player_stats['exp_vaep_value'] = player_stats['exp_vaep_value'].fillna(0)
player_stats['exp_offensive_value'] = player_stats['exp_offensive_value'].fillna(0)
player_stats['exp_defensive_value'] = player_stats['exp_defensive_value'].fillna(0)

match_stats = aggregate_player_to_match_stats(player_stats)

match_summary_stats = match_summary.merge(match_stats, how = "left", on = "Match_ID")

In [3]:
player_stats[player_stats['Match_ID'] == '202101_BrisbaneLions_Sydney'].tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
41,202101_BrisbaneLions_Sydney,Sydney,Sam Wicks,202101,Sam_Wicks,MEDIUM_FORWARD,CD_I1006232,22,180,80,45,RIGHT,NSW,2018.0,2020.0,Manly-Warringah (NSW)/Sydney (NEAFL),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-09-14,85.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,41.7,5.0,1.0,0.0,0.0,2.0,75.0,12.0,69.0,9.0,5.0,1.0,0.0,1.0,100.0,0.0,2.0,2.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,83.3,0.0,0.0,6.0,1.0,2.0,0.0,0.0,194.0,1.0,15.0,14.2,0.0,0.0,5.0,1.0,2.0,0.0,0.0,5.0,1.0,0.0,12.0,1.0,7.0,NaN,NaN,Half Forward Flank Left,FINAL_TEAM,Half-Forward,Forward,2021.0,0.0,2021,5.0964,0.0739,-0.0076,1.4309,1.5528,-0.1219,1.8600,2.6695,-0.8095,0.2936,-0.0012,0.1368,1.9963,Sam Wicks_2021,12.0,6.9036,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
42,202101_BrisbaneLions_Sydney,Sydney,Tom Hickey,202101,Tom_Hickey,RUCK,CD_I294266,30,202,96,31,LEFT,QLD,2010.0,2011.0,Morningside (Qld)/Gold Coast/St Kilda/West Coast,NaN,zone,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-03-06,82.0,0.0,0.0,31.0,5.0,3.0,0.0,0.0,3.0,0.0,60.9,14.0,1.0,0.0,0.0,7.0,82.6,23.0,92.0,19.0,10.0,0.0,2.0,3.0,0.0,1.0,0.0,9.0,11.0,24.0,6.0,25.0,42.9,5.0,1.0,5.0,83.3,0.0,0.0,12.0,1.1,3.0,0.0,0.0,400.0,6.0,11.0,19.0,2.0,56.0,7.0,3.0,0.0,5.0,4.0,1.0,0.0,9.0,23.0,6.0,9.0,Tom_Hickey,7.0,Ruck,FINAL_TEAM,Ruck,Ruck,2021.0,1.0,2021,0.0000,-0.4230,0.0300,2.3603,1.5111,0.8492,15.2474,10.0000,5.2475,-0.3704,0.0351,1.2706,9.7125,Tom Hickey_2021,0.0,0.0000,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
43,202101_BrisbaneLions_Sydney,Sydney,Tom McCartin,202101,Tom_McCartin,KEY_DEFENDER,CD_I1008198,22,193,95,30,RIGHT,VIC,2017.0,2018.0,"St Joseph&apos;s, Geel (Vic)/St Joseph&apos;s ...",33.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-12-30,87.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1.0,60.0,6.0,0.0,0.0,0.0,5.0,72.7,11.0,49.0,8.0,7.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,9.0,77.8,0.0,0.0,9.0,4.5,4.0,0.0,0.0,157.0,4.0,5.0,8.0,1.0,0.0,4.0,3.0,0.0,4.0,0.0,1.0,0.0,0.0,10.0,3.0,4.0,NaN,NaN,Full Back,FINAL_TEAM,Key-Back,Back,2021.0,0.0,2021,0.0000,-0.1081,0.0234,0.7347,0.1729,0.5618,4.8127,3.6471,1.1657,-0.4397,0.1014,0.5339,2.5907,Tom McCartin_2021,0.0,0.0000,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
44,202101_BrisbaneLions_Sydney,Sydney,Tom Papley,202101,Tom_Papley,MEDIUM_FORWARD,CD_I996765,25,177,79,11,RIGHT,VIC,2016.0,2016.0,Bunyip (Vic)/Gippsl

Glicko2

In [20]:
dayne_zorko = glicko2.Player()
dayne_zorko.rating, dayne_zorko.rd, dayne_zorko.vol

(1500.0, 350.0, 0.06)

In [18]:
player_stats[(player_stats['Match_ID'] == '202101_BrisbaneLions_Sydney') & (player_stats['Player'] == "Dayne Zorko")]['exp_vaep_value'].iloc[0]

11.6167738674

In [12]:
dayne_zorko = ttt.Player()
charlie_cameron = ttt.Player()
team_brisbane = [dayne_zorko, charlie_cameron]

tom_papley = ttt.Player()
will_hayward = ttt.Player()

team_sydney = [tom_papley, will_hayward]
team_brisbane, team_sydney

([Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030),
  Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030)],
 [Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030),
  Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030)])

In [13]:
match = ttt.Game([team_sydney, team_brisbane]) # Brisbane lost, Sydney Won
round(match.evidence, 3), match.likelihoods

(0.5,
 ([N(mu=15.247, sigma=14.017), N(mu=15.247, sigma=14.017)],
  [N(mu=-15.247, sigma=14.017), N(mu=-15.247, sigma=14.017)]))

In [16]:
match.likelihoods

([N(mu=15.247, sigma=14.017), N(mu=15.247, sigma=14.017)],
 [N(mu=-15.247, sigma=14.017), N(mu=-15.247, sigma=14.017)])

In [18]:
match.posteriors()

[[N(mu=2.361, sigma=5.516), N(mu=2.361, sigma=5.516)],
 [N(mu=-2.361, sigma=5.516), N(mu=-2.361, sigma=5.516)]]

In [17]:
zorko_posterior, cameron_posterior = match.posteriors()[0][0], match.posteriors()[0][1]
sydney_posterior, brisbane_posterior

(N(mu=2.361, sigma=5.516), N(mu=-2.361, sigma=5.516))

In [30]:
match.performance(0)

N(mu=0.000, sigma=6.083)

TrueSkill

In [31]:
import trueskill

In [76]:
brisbane = trueskill.Rating()
sydney = trueskill.Rating()
brisbane, sydney

(trueskill.Rating(mu=25.000, sigma=8.333),
 trueskill.Rating(mu=25.000, sigma=8.333))

In [77]:
print('{:.1%} chance to draw'.format(trueskill.quality_1vs1(brisbane, sydney)))

44.7% chance to draw


In [81]:
new_sydney, new_brisbane = trueskill.rate_1vs1(sydney, brisbane)
new_brisbane, new_sydney

(trueskill.Rating(mu=20.604, sigma=7.171),
 trueskill.Rating(mu=29.396, sigma=7.171))

In [82]:
import itertools
import math

def win_probability(team1, team2):
    delta_mu = team1.mu - team2.mu
    sum_sigma = (team1.sigma ** 2)+ (team2.sigma ** 2)
    denom = math.sqrt((BETA * BETA) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)

In [83]:
BETA = 25/6

In [84]:
win_probability(brisbane, sydney), win_probability(sydney, brisbane)

(0.5000000150000002, 0.5000000150000002)

In [85]:
win_probability(new_brisbane, new_sydney), win_probability(new_sydney, new_brisbane)

(0.2113266125927094, 0.7886733874072906)